In [ ]:
!gdown --id 1oWqGyirJe85zUnrwX3BUxKUTQpxzXCEJ -O Training e test set Umberto.zip

import os
os.mkdir('/content/Training e test set Umberto')
!unzip /content/Training e test set Umberto.zip -d /content/Training e test set Umberto/


In [1]:

import pathlib
import pandas as pd
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0";

!pip install ktrain
import ktrain
from ktrain import text
MODEL_NAME="Musixmatch/umberto-commoncrawl-cased-v1"

     |████████████████████████████████| 25.3 MB 5.2 MB/s eta 0:00:01    |▏                               | 122 kB 595 kB/s eta 0:00:43     |████▍                           | 3.4 MB 595 kB/s eta 0:00:37     |██████████████████████          | 17.3 MB 595 kB/s eta 0:00:14
     |████████████████████████████████| 981 kB 49.9 MB/s eta 0:00:01
     |████████████████████████████████| 263 kB 55.0 MB/s eta 0:00:01
     |████████████████████████████████| 468 kB 41.7 MB/s eta 0:00:01
  Created wheel for ktrain: filename=ktrain-0.28.3-py3-none-any.whl size=25292659 sha256=46250e20b541e75ce62c2ad142394cad8b53fea16ddd7169bc25f5fb71ddb4d4
  Stored in directory: /root/.cache/pip/wheels/6a/7e/c3/f46cdfc2b81c54424923b1405d7e670c35cacc11ada9a47b1c
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9929 sha256=a2c3fcdbe052e43a61d05567f7fb285eb1c3a8e49d50a50767e4e82bfab5fd01
  Stored in directory: /root/.cache/pip/wheels/f5/ac/f1/4e13d7aff05c722d142b7d20a88ad63f9aab11b895411241a4
  C

In [2]:
train_path= "../input/dati-di-training-e-di-test/haspeede2_dev_taskAB.csv"
test_path="../input/dati-di-training-e-di-test/haspeede2_reference_taskAB-tweets.csv"
tr_path=pathlib.Path(train_path)
te_path=pathlib.Path(test_path)
if tr_path.exists():
    print("Train data path set.")
else:
    raise SystemExit("Training Data Path does not exist.")

if te_path.exists():
    print("Test data path set.")
else:
    raise SystemExit("Test Data Path does not exist.")

Train data path set.
Test data path set.


In [3]:
train_df= pd.read_csv(train_path, encoding='utf-8', sep="\t", header=None)
train_df=train_df.drop([0])
train_df.head()

,0,1,2,3
1,2066,"È terrorismo anche questo, per mettere in uno ...",0,0
2,2045,@user @user infatti finché ci hanno guadagnato...,0,0
3,61,"Corriere: Tangenti, Mafia Capitale dimenticata...",0,0
4,1259,"@user ad uno ad uno, perché quando i migranti ...",0,0
5,949,Il divertimento del giorno? Trovare i patrioti...,0,0


In [4]:
test_df= pd.read_csv(test_path, encoding='utf-8', sep="\t",header=None)
test_df.head()

,0,1,2,3
0,11834,@user A me pare una scelta politica suicida pu...,1,0
1,12113,@user e' un perfetto musulmano!!! chi dice il ...,1,1
2,11770,Mai Tg e i giornaloni hanno parlato di questa ...,1,1
3,11937,@user Ipocriti farabutti. Fanno morire i terre...,1,1
4,11870,@user @user @user L'IMMIGRAZIONE C'E' STATO UN...,1,1


In [5]:
x_train=train_df[1].tolist()
y_train=train_df[2].tolist()
x_test=test_df[1].tolist()
y_test=test_df[2].tolist()

In [6]:
from transformers import AutoModel, AutoTokenizer
umberto = AutoModel.from_pretrained("Musixmatch/umberto-commoncrawl-cased-v1")
tokenizerU = AutoTokenizer.from_pretrained("Musixmatch/umberto-commoncrawl-cased-v1")

from transformers import TrainingArguments
training_args = TrainingArguments(
    "test_trainer",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    learning_rate=2e-4
)

from transformers import Trainer
trainer = Trainer(
    model=umberto, 
    args=training_args, 
    train_dataset=train_df, 
    eval_dataset=test_df,
    tokenizer=tokenizerU
)
trainer.train()



Downloading:   0%|          | 0.00/508 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/794k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.68M [00:00<?, ?B/s]

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: wandb version 0.12.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade



CondaEnvException: Unable to determine environment

Please re-run this command with one of the following options:

* Provide an environment name via --name or -n
* Re-run this command inside an activated conda environment.



KeyError: 2971